In [1]:
import re
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import argparse
import os
import warnings
import pandas as pd
from fpdf import FPDF
import cairosvg
from IPython.display import SVG, display
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'


/home1/zhuyixin/.conda/envs/assembly/lib/python3.10/site-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [2]:
species = "mPonAbe1"
haploid = False
gene = "IGH"
AssType = "hap2"
dirOut = f"/home1/zhuyixin/zhuyixin_proj/AssmQuality/errorPlots/{species}"
dirStat = f"/home1/zhuyixin/zhuyixin_proj/AssmQuality/errorStats/{species}"
bam_file_path = f"/home1/zhuyixin/zhuyixin_proj/AssmQuality/aligned_bam/{species}/{species}_merged_sorted_primary.bam"
if (AssType == "mat") or (AssType == "hap1"):
    atype = "pri"
else:
    atype = "alt"
pri_pileup_file = f'../errorStats/{species}/{gene}_{atype}_pileup.txt'


In [3]:
def parse_read_bases(read_bases):
    """
    Parse read bases to count correct matches (., and ,) while correctly handling indels,
    read starts (^), read ends ($), and skipping over sequences indicating indels (+nXXX or -nXXX).
    """
    cleaned_bases = re.sub(r'\^.', '', read_bases)
    cleaned_bases = cleaned_bases.replace('$', '')
    cleaned_bases = re.sub(r'[\+\-](\d+)([ACGTNacgtn]+)', '', cleaned_bases)
    cleaned_bases = cleaned_bases.replace('*', '')
    correct = cleaned_bases.count('.') + cleaned_bases.count(',')

    return correct


import re

def count_indels(read_bases):
    """
    Count the number of indels in the read bases, handling both insertions and deletions.
    Indels are indicated by + or - followed by the length of the indel and the sequence.
    """

    # Find all occurrences of indels, both insertions and deletions
    indels = re.findall(r'[\+\-](\d+)([ACGTNacgtn]+)', read_bases)

    # The total count of indels is the sum of the occurrences
    indel_count = sum(int(length) for length, _ in indels)

    return indel_count

In [103]:
results_pri = {}  # To store results, keyed by (chromosome, position)
with open(pri_pileup_file, 'r') as f:
    for line in f:
        chrom, pos, ref_base, depth, read_bases, _ = line.split()[:6]
        correct = parse_read_bases(read_bases)
        indel = count_indels(read_bases)            
        if int(depth) > 0:
            percent_correct = (correct / int(depth)) * 100
        else:
            percent_correct = 0
         # Store or process results
        results_pri[(chrom, pos)] = (correct, percent_correct, depth, indel)

pri_p = [(*key, *value) for key, value in results_pri.items()]
pri_pileup = pd.DataFrame(pri_p, columns=['Chrom', 'Pos', 'Correct', 'PercentCorrect', "Depth", "Indel"])
pri_pileup['Pos'] = pd.to_numeric(pri_pileup['Pos'])
pri_pileup['Correct'] = pd.to_numeric(pri_pileup['Correct'])
pri_pileup['PercentCorrect'] = pd.to_numeric(pri_pileup['PercentCorrect'])
pri_pileup['Depth'] = pd.to_numeric(pri_pileup['Depth'])
pri_pileup['Indel'] = pd.to_numeric(pri_pileup['Indel'])
pri_pileup.sort_values(by=['Chrom', 'Pos'], inplace=True)
pri_pileup.describe()

,Pos,Correct,PercentCorrect,Depth,Indel
count,1.215789e+06,1.215789e+06,1.215789e+06,1.215789e+06,1.215789e+06
mean,1.031147e+08,4.822201e+01,9.989388e+01,4.827318e+01,1.117924e-01
std,3.509682e+05,8.167864e+00,5.941782e-01,8.171534e+00,1.989873e+00
min,1.025068e+08,2.200000e+01,6.111111e+01,2.400000e+01,0.000000e+00
25%,1.028107e+08,4.300000e+01,1.000000e+02,4.300000e+01,0.000000e+00
50%,1.031147e+08,4.800000e+01,1.000000e+02,4.800000e+01,0.000000e+00
75%,1.034186e+08,5.400000e+01,1.000000e+02,5.400000e+01,0.000000e+00
max,1.037226e+08,7.500000e+01,1.000000e+02,7.500000e+01,1.861000e+03


In [104]:
pri_pileup[pri_pileup['PercentCorrect']> 99].shape[0]/pri_pileup.shape[0]

0.9590159147681053

In [105]:
igh = pd.read_csv(f"../errorStats/{species}/{gene}.txt", sep="\t", names=['read_name','chromosome','start','read_length','mapping_quality','mismatches','mismatch_rate','longindels','total_indel_length','indel_rate','soft_clipping','hard_clipping'])
igh["start"] = pd.to_numeric(igh["start"])
igh["read_length"] = pd.to_numeric(igh["read_length"])
igh["mapping_quality"] = pd.to_numeric(igh["mapping_quality"])
igh['mismatches'] = igh['mismatches'].astype('int')
igh['mismatch_rate'] = igh['mismatch_rate'].astype('float')
igh['longindels'] = igh['longindels'].astype('int')
igh['total_indel_length'] = igh['total_indel_length'].astype('int')
igh['indel_rate'] = igh['indel_rate'].astype('float')
igh['soft_clipping'] = igh['soft_clipping'].astype('int')
igh['hard_clipping'] = igh['hard_clipping'].astype('int')
igh['end'] = igh['start'] + igh['read_length'] - 1
chr1 = pri_pileup['Chrom'].value_counts().index[0]
igh_pri = igh[(igh['chromosome'] == chr1)]
igh_pri

,read_name,chromosome,start,read_length,mapping_quality,mismatches,mismatch_rate,longindels,total_indel_length,indel_rate,soft_clipping,hard_clipping,end
3866,m54329U_220313_173323/117639354/ccs,chr15_hap2_hsa14,102481413,28540,30,5,0.000175,0,0,0.000000,3,0,102509952
3867,m54329U_210404_020346/15336368/ccs,chr15_hap2_hsa14,102487172,19795,15,0,0.000000,0,0,0.000000,0,0,102506966
3868,m54329U_220313_173323/76744132/ccs,chr15_hap2_hsa14,102487651,28758,60,4,0.000139,0,0,0.000000,0,0,102516408
3869,m64076_220312_125202/28902388/ccs,chr15_hap2_hsa14,102487840,21075,17,0,0.000000,0,0,0.000000,0,0,102508914
3870,m64076_220313_224245/157485750/ccs,chr15_hap2_hsa14,102488166,19875,20,41,0.002063,3,11,0.000553,0,0,102508040
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7358,m64076_210330_204128/60688176/ccs,chr15_hap2_hsa14,103717856,11074,4,0,0.000000,0,0,0.000000,0,0,103728929
7359,m64076_210219_011735/107479947/ccs,chr15_hap2_hsa14,103718106,11360,5,7,0.000616,0,0,0.000000,0,0,103729465
7360,m54329U_220227_232630/122227393/ccs,chr15_hap2_hsa14,103720334,17839,60,1,0.000056,2,16,0.000897,0,0,103738172
7361,m64076_210221_195503/62851886/ccs,chr15_hap2_hsa14,103720910,10529,23,0,0.000000,0,0,0.000000,0,0,103731438


In [106]:
geneFile = f'../primate_igtr/{species}_{AssType}_{gene}.csv'
genes = pd.read_csv(geneFile)
genes['SeqLength'] = genes['Sequence'].apply(len)
genes['EndPos'] = genes['Pos'] + genes['SeqLength'] - 1 
genes.head(10)

,LocusPos,Locus,GeneType,Contig,Pos,Strand,Sequence,Productive,SeqLength,EndPos
0,20000,IGH,J,chr15_hap2_hsa14,102516796,-,ATTACTACGGTATGGACTTCTGGGACTAAGGGACCATGGTCACCGT...,NaN,63,102516858
1,21021,IGH,J,chr15_hap2_hsa14,102517817,-,ACTACTTTGACTACTGGGGCCAGGGAACCCTGCTCACCGTCTCCTCAG,NaN,48,102517864
2,21393,IGH,J,chr15_hap2_hsa14,102518189,-,TGATGCTTTTGATATCTGGGGCCTAGGGGTCGTGGTCACCGTCTCT...,NaN,50,102518238
3,22005,IGH,J,chr15_hap2_hsa14,102518801,-,CTACTGGTACTTCGATCTCTGGGGCCGTGGCACCCTGGTCACCGTC...,NaN,53,102518853
4,22213,IGH,J,chr15_hap2_hsa14,102519009,-,GCCGAATACTTCAAGCACTGGGGCCAGGGCATCCTGGTCACCGTCT...,NaN,52,102519060
5,22356,IGH,D,chr15_hap2_hsa14,102519152,+,TCCCCAGTTAG,NaN,11,102519162
6,44167,IGH,D,chr15_hap2_hsa14,102540963,-,GGTATAGTGGGAGCTACTAC,NaN,20,102540982
7,44674,IGH,D,chr15_hap2_hsa14,102541470,+,GTAGCCACTGCTATACCC,NaN,18,102541487
8,44674,IGH,D,chr15_hap2_hsa14,102541470,-,GGGTATAGCAGTGGCTAC,NaN,18,102541487
9,51433,IGH,D,chr15_hap2_hsa14,102548229,-,AGGATATTGTAGTGGTGGTAGCTGCTACGCC,NaN,31,102548259


In [107]:
for index, row in genes.iterrows():
    chrom = row['Contig']
    start = row['Pos']
    length = row['SeqLength']
    end = row['EndPos']
    locus = row['Locus']
    geneType = row['GeneType']
    gene_df = pri_pileup[(pri_pileup['Pos'] >= start) & (pri_pileup['Pos'] <= end)]
    positions_with_10x = (gene_df['Depth'] >= 10).sum()
    gene_df['mismatch_rate'] = 100 - gene_df['PercentCorrect']
    mismatchPos = (gene_df['mismatch_rate'] > 20).sum()
    matchPos = (gene_df['mismatch_rate'] < 20).sum()
    reads_spanning_region = igh_pri[(igh_pri['start'] <= start) & (igh_pri['end'] >= end)].shape[0]
    Fully_Spanning_Reads_100 = igh_pri[(igh_pri['start'] <= start) & (igh_pri['end'] >= end) & (igh_pri['mismatches'] == 0)].shape[0]
    if length > 0:
        Average_Coverage = reads_spanning_region / length
        percent_accuracy = (matchPos / length) * 100
    else:
        Average_Coverage = 0
        percent_accuracy = 0
    Position_Mismatches = ';'.join(map(str, gene_df['Depth']-gene_df['Correct']))
    Position_matches = ';'.join(map(str, gene_df['Correct']))
    genes.loc[index, 'Average_Coverage'] = Average_Coverage
    genes.loc[index, 'Mismatched_Positions'] = mismatchPos
    genes.loc[index, 'Matched_Positions'] = matchPos
    genes.loc[index, 'Position_Mismatches'] = Position_Mismatches
    genes.loc[index, 'Position_Matches'] = Position_matches
    genes.loc[index, 'Percent_Accuracy'] = percent_accuracy
    genes.loc[index, 'Positions_With_At_Least_10x_Coverage'] = positions_with_10x
    genes.loc[index, 'Fully_Spanning_Reads'] = reads_spanning_region
    genes.loc[index, 'Fully_Spanning_Reads_100%_Match'] = Fully_Spanning_Reads_100

genes    

,LocusPos,Locus,GeneType,Contig,Pos,Strand,Sequence,Productive,SeqLength,EndPos,Average_Coverage,Mismatched_Positions,Matched_Positions,Position_Mismatches,Position_Matches,Percent_Accuracy,Positions_With_At_Least_10x_Coverage,Fully_Spanning_Reads,Fully_Spanning_Reads_100%_Match
0,20000,IGH,J,chr15_hap2_hsa14,102516796,-,ATTACTACGGTATGGACTTCTGGGACTAAGGGACCATGGTCACCGT...,NaN,63,102516858,0.666667,0.0,63.0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,42;42;42;42;42;42;42;42;42;42;42;42;42;42;42;4...,100.0,63.0,42.0,29.0
1,21021,IGH,J,chr15_hap2_hsa14,102517817,-,ACTACTTTGACTACTGGGGCCAGGGAACCCTGCTCACCGTCTCCTCAG,NaN,48,102517864,0.791667,0.0,48.0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,38;38;38;38;38;38;38;38;38;38;38;38;38;38;38;3...,100.0,48.0,38.0,24.0
2,21393,IGH,J,chr15_hap2_hsa14,102518189,-,TGATGCTTTTGATATCTGGGGCCTAGGGGTCGTGGTCACCGTCTCT...,NaN,50,102518238,0.760000,0.0,50.0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,38;38;38;38;38;38;38;39;39;39;39;39;39;39;39;3...,100.0,50.0,38.0,24.0
3,22005,IGH,J,chr15_hap2_hsa14,102518801,-,CTACTGGTACTTCGATCTCTGGGGCCGTGGCACCCTGGTCACCGTC...,NaN,53,102518853,0.754717,0.0,53.0,1;0;0;0;0;0;0;0;0;0;0;1;0;0;0;0;0;0;0;1;0;0;0;...,39;40;40;40;40;40;40;40;40;40;40;39;40;40;40;4...,100.0,53.0,40.0,26.0
4,22213,IGH,J,chr15_hap2_hsa14,102519009,-,GCCGAATACTTCAAGCACTGGGGCCAGGGCATCCTGGTCACCGTCT...,NaN,52,102519060,0.750000,0.0,52.0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,39;39;39;39;39;39;39;39;39;39;39;39;39;39;39;3...,100.0,52.0,39.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1124922,IGH,V,chr15_hap2_hsa14,103621718,-,CAGGTCACCTTGAAGGAGTCTGGTCCTGCACTGGTGAAACCCACAC...,True,301,103622018,0.156146,0.0,301.0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,47;47;47;47;47;47;47;47;47;47;47;47;47;47;47;4...,100.0,301.0,47.0,30.0
92,1135248,IGH,V,chr15_hap2_hsa14,103632044,-,GAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCTGGGG...,True,302,103632345,0.112583,0.0,302.0,0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;0;0;0;0;0;0;0;0;...,38;38;38;38;38;38;38;38;38;38;38;38;38;37;38;3...,100.0,302.0,34.0,24.0
93,1152871,IGH,V,chr15_hap2_hsa14,103649667,-,GAGGTGCAGCTGTTGGAGTCAGGGGGAGGCTTGGTACAGCCTGAGG...,True,296,103649962,0.165541,0.0,296.0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,50;50;50;50;50;50;50;50;50;50;50;50;50;50;50;5...,100.0,296.0,49.0,36.0
94,1194890,IGH,V,chr15_hap2_hsa14,103691686,-,GAGGTGCAGCTGGTGCAGTCTGCAGCAGAGGTGAAAAGGCCCGGGG...,True,294,103691979,0.190476,0.0,294.0,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,56;56;56;56;56;56;56;56;56;56;56;56;56;56;56;5...,100.0,294.0,56.0,43.0


In [108]:
genes.describe()

,LocusPos,Pos,SeqLength,EndPos,Average_Coverage,Mismatched_Positions,Matched_Positions,Percent_Accuracy,Positions_With_At_Least_10x_Coverage,Fully_Spanning_Reads,Fully_Spanning_Reads_100%_Match
count,9.600000e+01,9.600000e+01,96.000000,9.600000e+01,96.000000,96.0,96.000000,96.0,96.000000,96.000000,96.000000
mean,4.578251e+05,1.029546e+08,189.697917,1.029548e+08,0.768027,0.0,189.697917,100.0,189.697917,43.208333,26.791667
std,3.997137e+05,3.997137e+05,133.755088,3.998215e+05,0.906760,0.0,133.755088,0.0,133.755088,8.750238,6.425142
min,2.000000e+04,1.025168e+08,9.000000,1.025169e+08,0.108108,0.0,9.000000,100.0,9.000000,30.000000,15.000000
25%,7.221600e+04,1.025690e+08,24.000000,1.025690e+08,0.157142,0.0,24.000000,100.0,24.000000,35.750000,21.000000
50%,3.426625e+05,1.028395e+08,293.500000,1.028398e+08,0.185176,0.0,293.500000,100.0,293.500000,43.500000,26.000000
75%,8.542465e+05,1.033510e+08,296.000000,1.033513e+08,1.395161,0.0,296.000000,100.0,296.000000,50.000000,31.000000
max,1.215788e+06,1.037126e+08,308.000000,1.037129e+08,4.888889,0.0,308.000000,100.0,308.000000,72.000000,43.000000


In [109]:
outFile = f"{dirStat}/{species}.{AssType}.{gene}.csv"
genes = genes.iloc[:, 1:]
genes.to_csv(outFile, sep='\t')


## combined csv

In [114]:
species = "mPonAbe1"
AssType = "hap2"
dirStat = f"/home1/zhuyixin/zhuyixin_proj/AssmQuality/errorStats/{species}"

In [115]:
csv_files = [f for f in os.listdir(dirStat) if AssType in f and f.endswith('.csv')]
order = ['IGH', 'IGK', 'IGL', 'TRA', 'TRB', 'TRG']
sort_order = {key: i for i, key in enumerate(order)}
sorted_filenames = sorted(csv_files, key=lambda x: sort_order[x.split('.')[2]])
dataframes = [pd.read_csv(f"{dirStat}/{filename}", sep = "\t") for filename in sorted_filenames]
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df = combined_df.iloc[:, 0:]
outFile = f"{dirStat}/{species}.{AssType}.combined.csv"
combined_df.to_csv(outFile, sep='\t')


In [116]:
combined_df.describe()

,Unnamed: 0,Pos,SeqLength,EndPos,Average_Coverage,Mismatched_Positions,Matched_Positions,Percent_Accuracy,Positions_With_At_Least_10x_Coverage,Fully_Spanning_Reads,Fully_Spanning_Reads_100%_Match
count,281.000000,2.810000e+02,281.000000,2.810000e+02,281.000000,281.0,281.000000,281.0,281.000000,281.000000,281.000000
mean,30.836299,8.006739e+07,250.451957,8.006764e+07,0.388673,0.0,250.451957,100.0,250.451957,47.494662,29.911032
std,23.911927,4.805310e+07,94.588009,4.805308e+07,0.602992,0.0,94.588009,0.0,94.588009,9.160521,6.789376
min,0.000000,1.708122e+07,9.000000,1.708150e+07,0.093333,0.0,9.000000,100.0,9.000000,28.000000,13.000000
25%,12.000000,2.265777e+07,280.000000,2.265806e+07,0.153310,0.0,280.000000,100.0,280.000000,41.000000,26.000000
50%,26.000000,1.025608e+08,287.000000,1.025608e+08,0.174377,0.0,287.000000,100.0,287.000000,47.000000,30.000000
75%,44.000000,1.035571e+08,296.000000,1.035574e+08,0.206897,0.0,296.000000,100.0,296.000000,54.000000,34.000000
max,95.000000,1.445671e+08,317.000000,1.445674e+08,4.888889,0.0,317.000000,100.0,317.000000,74.000000,51.000000


In [19]:
species = "mPonPyg2"
AssType = "hap2"
dirStat = f"/home1/zhuyixin/zhuyixin_proj/AssmQuality/errorStats/{species}"
order = ['IGH', 'IGK', 'IGL', 'TRA', 'TRB', 'TRG']


In [20]:
if (AssType == "mat") or (AssType == "hap1"):
    atype = "pri"
else:
    atype = "alt"
for gene in order:
    pri_pileup_file = f'../errorStats/{species}/{gene}_{atype}_pileup.txt'
    results_pri = {}  # To store results, keyed by (chromosome, position)
    with open(pri_pileup_file, 'r') as f:
        for line in f:
            chrom, pos, ref_base, depth, read_bases, _ = line.split()[:6]
            correct = parse_read_bases(read_bases)
            indel = count_indels(read_bases)            
            if int(depth) > 0:
                percent_correct = (correct / int(depth)) * 100
            else:
                percent_correct = 0
             # Store or process results
            results_pri[(chrom, pos)] = (correct, percent_correct, depth, indel)
    
    pri_p = [(*key, *value) for key, value in results_pri.items()]
    pri_pileup = pd.DataFrame(pri_p, columns=['Chrom', 'Pos', 'Correct', 'PercentCorrect', "Depth", "Indel"])
    pri_pileup['Pos'] = pd.to_numeric(pri_pileup['Pos'])
    pri_pileup['Correct'] = pd.to_numeric(pri_pileup['Correct'])
    pri_pileup['PercentCorrect'] = pd.to_numeric(pri_pileup['PercentCorrect'])
    pri_pileup['Depth'] = pd.to_numeric(pri_pileup['Depth'])
    pri_pileup['Indel'] = pd.to_numeric(pri_pileup['Indel'])
    pri_pileup.sort_values(by=['Chrom', 'Pos'], inplace=True)
    print(gene)
    print(pri_pileup['Depth'].mean(), pri_pileup[pri_pileup['PercentCorrect']> 80].shape[0]/pri_pileup.shape[0])

IGH
29.557833800636402 0.9999759207310466
IGK
33.188198020410596 0.9999833518810454
IGL
30.192151540724858 0.9999460195090056
TRA
31.91613480894697 0.9999797194387152
TRB
32.27359826284186 0.9999806614074904
TRG
28.858019833149694 0.9999790125400073
